In [1]:
%matplotlib inline

from random import randint
import matplotlib
import numpy as np
import torch

In [10]:
# load pre-trained model
# on GPU (encoding ~1000 sentences/s, default)
model = torch.load('..//SentenceEmbedding/infersent.allnli.pickle')
# on CPU (~40 sentences/s)
# model = torch.load('infersent.allnli.pickle', map_location=lambda storage, loc: storage)

C:\Users\vbigm\Anaconda3\lib\site-packages\torch\serialization.py:286: SourceChangeWarning: source code of class 'models.BLSTMEncoder' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


In [11]:
# set path to word vectors from glove
GLOVE_PATH = 'D:/DataSet/NLP_GLOVE/Pre-Trained Vectors/glove840B300d/glove.840B.300d.txt'
#GLOVE_PATH = '../SemEval/glove_6B_50d.txt'
model.set_glove_path(GLOVE_PATH)

# load the K most common english words
model.build_vocab_k_words(K=100000)

Vocab size : 100000


In [12]:
# Load input
sentences = []
with open('STS.input.track5.en-en.txt', encoding="utf-8") as f:
    for line in f:
        sentences.append(line.strip())
print(len(sentences))

250


In [29]:
sent1 = []
sent2 = []
for line in sentences:
    [s1, s2] = line.split("\t")
    sent1.append(s1)
    sent2.append(s2)

In [40]:
list(zip(sent1[:5],sent2[:5]))

[('A person is on a baseball team.',
  'A person is playing basketball on a team.'),
 ('Our current vehicles will be in museums when everyone has their own aircraft.',
  'The car needs to some work'),
 ('A woman supervisor is instructing the male workers.',
  'A woman is working as a nurse.'),
 ('A bike is next to a couple women.', 'A child next to a bike.'),
 ('The group is eating while taking in a breathtaking view.',
  'A group of people take a look at an unusual tree.')]

In [47]:
len(sent1)

250

In [43]:
embeddings1 = model.encode(sent1, bsize=128, tokenize=False, verbose=True)
embeddings2 = model.encode(sent2, bsize=128, tokenize=False, verbose=True)

Nb words kept : 2446/2668 (91.68 %)


C:\Users\vbigm\Dropbox\Course Stony Brook\17 Fall\CSE593 Independent Study\Jason\SentenceTexualSimilarity\models.py:53: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greately increasing memory usage. To compact weights again call flatten_parameters().
  sent_output = self.enc_lstm(sent_packed)[0]  # seqlen x batch x 2*nhid


Speed : 836.48 sentences/s (gpu mode, bsize=128)
Nb words kept : 2446/2682 (91.2 %)
Speed : 892.27 sentences/s (gpu mode, bsize=128)


In [57]:
print(embeddings1.shape)
print(embeddings2.shape)

(250, 4096)
(250, 4096)


In [48]:
def cosine(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

In [53]:
def similarity(s1, s2):
    return np.nan_to_num(cosine(np.nan_to_num(s1), np.nan_to_num(s2)))

In [62]:
similar = [ similarity(embeddings1[i], embeddings2[i]) for i in range(len(embeddings1))]

In [68]:
print(len(similar))
similar[:5]

250


[0.8024776, 0.67585117, 0.75681674, 0.59188825, 0.68153501]

In [64]:
from scipy.stats import spearmanr, pearsonr

In [76]:
# Load gold standard
gs = []
with open('STS.gs.track5.en-en.txt', encoding="utf-8") as f:
    for line in f:
        gs.append(float(line.strip()))
print(len(gs))

250


In [77]:
pearsonr(similar, gs)

(0.63042565601845013, 4.1917590051526955e-29)

In [78]:
np.corrcoef(gs, similar)

array([[ 1.        ,  0.63042568],
       [ 0.63042568,  1.        ]])